In [1]:
import os
from scipy.io import loadmat
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# --- Chemins ---
raw_data_dir = "../data/raw/"
processed_images_dir = "../data/processed/Images"  # Ajuste en fonction de ton dossier de données

# --- Lecture des fichiers .mat ---
train_mat = loadmat(os.path.join(raw_data_dir, "train_list.mat"))
test_mat = loadmat(os.path.join(raw_data_dir, "test_list.mat"))

train_list = [item[0][0] for item in train_mat["file_list"]]
test_list = [item[0][0] for item in test_mat["file_list"]]

print(f"Exemple d'image dans train: {train_list[0]}")

# --- Préparer les générateurs de données ---
datagen = ImageDataGenerator(rescale=1./255)  # Normalisation des pixels sans augmentation

# --- Générateur d'entraînement ---
train_generator = datagen.flow_from_directory(
    processed_images_dir,  # Dossier contenant les images traitées
    target_size=(224, 224),  # Taille des images
    batch_size=32,
    class_mode='categorical'  # Classification multiclasse
)

# --- Générateur de test ---
test_generator = datagen.flow_from_directory(
    processed_images_dir,  # Dossier contenant les images de test
    target_size=(224, 224),  # Taille des images
    batch_size=32,
    class_mode='categorical',  # Classification multiclasse
    shuffle=False  # On ne mélange pas les images de test
)

# --- Définition du modèle CNN ---
model = models.Sequential()

# Première couche de convolution
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# Deuxième couche de convolution
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Troisième couche de convolution
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Aplatir les résultats pour la couche fully connected
model.add(layers.Flatten())

# Couche fully connected
model.add(layers.Dense(128, activation='relu'))

# Couche de sortie
model.add(layers.Dense(len(train_generator.class_indices), activation='softmax'))  # Nombre de classes

# --- Compilation du modèle ---
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --- Callback pour la sauvegarde du meilleur modèle ---
checkpoint_callback = ModelCheckpoint('best_model.h5', 
                                      save_best_only=True, 
                                      monitor='val_loss', 
                                      mode='min', 
                                      verbose=1)

# --- Entraînement du modèle ---
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_list) // 32,  # Nombre de batches par époque
    epochs=20,
    validation_data=test_generator,  # Validation sur les images de test
    validation_steps=len(test_list) // 32,  # Nombre de batches pour la validation
    callbacks=[checkpoint_callback]
)

# --- Évaluation du modèle ---
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_list) // 32)
print(f"Test accuracy: {test_acc}")

Exemple d'image dans train: n02085620-Chihuahua/n02085620_5927.jpg
Found 20580 images belonging to 120 classes.
Found 20580 images belonging to 120 classes.


c:\Users\Kraich\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Kraich\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - accuracy: 0.0098 - loss: 4.7967
Epoch 1: val_loss improved from inf to 4.74002, saving model to best_model.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 118s 311ms/step - accuracy: 0.0098 - loss: 4.7967 - val_accuracy: 0.0205 - val_loss: 4.7400
Epoch 2/20
269/375 ━━━━━━━━━━━━━━━━━━━━ 26s 251ms/step - accuracy: 0.0201 - loss: 4.6954

c:\Users\Kraich\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


...

375/375 ━━━━━━━━━━━━━━━━━━━━ 94s 250ms/step - accuracy: 0.9843 - loss: 0.0968 - val_accuracy: 0.9833 - val_loss: 0.0952
Epoch 15/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.9897 - loss: 0.0747
Epoch 15: val_loss improved from 0.09516 to 0.06465, saving model to best_model.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 110s 294ms/step - accuracy: 0.9897 - loss: 0.0747 - val_accuracy: 0.9911 - val_loss: 0.0646
Epoch 16/20
269/375 ━━━━━━━━━━━━━━━━━━━━ 26s 249ms/step - accuracy: 0.9944 - loss: 0.0338
Epoch 16: val_loss did not improve from 0.06465
375/375 ━━━━━━━━━━━━━━━━━━━━ 89s 236ms/step - accuracy: 0.9941 - loss: 0.0349 - val_accuracy: 0.9899 - val_loss: 0.0691
Epoch 17/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.9926 - loss: 0.0513
Epoch 17: val_loss improved from 0.06465 to 0.06449, saving model to best_model.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 114s 305ms/step - accuracy: 0.9926 - loss: 0.0513 - val_accuracy: 0.9915 - val_loss: 0.0645
Epoch 18/20
269/375 ━━━━━━━━━━━━━━━━━━━━ 25s 244ms/step - accuracy: 0.9898 - loss: 0.0568
Epoch 18: val_loss improved from 0.06449 to 0.05468, saving model to best_model.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 86s 230ms/step - accuracy: 0.9903 - loss: 0.0559 - val_accuracy: 0.9918 - val_loss: 0.0547
Epoch 19/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.9878 - loss: 0.0590
Epoch 19: val_loss did not improve from 0.05468
375/375 ━━━━━━━━━━━━━━━━━━━━ 110s 294ms/step - accuracy: 0.9877 - loss: 0.0591 - val_accuracy: 0.9886 - val_loss: 0.0647
Epoch 20/20
269/375 ━━━━━━━━━━━━━━━━━━━━ 25s 239ms/step - accuracy: 0.9885 - loss: 0.0596
Epoch 20: val_loss improved from 0.05468 to 0.05343, saving model to best_model.h5


375/375 ━━━━━━━━━━━━━━━━━━━━ 85s 226ms/step - accuracy: 0.9877 - loss: 0.0620 - val_accuracy: 0.9914 - val_loss: 0.0534
268/268 ━━━━━━━━━━━━━━━━━━━━ 21s 77ms/step - accuracy: 0.9915 - loss: 0.0566
Test accuracy: 0.9913712739944458
